### Simplex Tableau
---

Construção do algoritmo Simplex na forma Tableau:

* Não precisamos dos passos dados no Simplex "raiz", as iterações são compostas apenas de operações matriciais
* No Tableau, criamos uma matriz indexada pelas variábeis básicas, composta pela matriz de restrições com o vetor $b$ concatenado, e o vetor de custo com o resultado $z$ concatenado, como mostra a figura abaixo:

<img src="files/simplex_tableau.jpg">

### Implementação

In [240]:
import numpy as np

In [417]:
def SimplexTableau(A, b, c, B, w=0, max_counter=100):
    
    T = np.concatenate((np.zeros(A.shape[0]).reshape(A.shape[0],1), A, b), axis=1)
    T = np.concatenate((np.array([1] + list(-c.T[0]) + [0]).reshape(1, c.shape[0]+2), T), axis=0)
    
    counter = 0

    while np.any(T[0,:] < 0) and (counter < max_counter):

        print('>>> Iteração {}:'.format(counter))
        j = np.where(T[0,1:-1] < 0)[0][0] # primeiro elemento em quem -c > 0 (exceto primeira e última colunas)
        j = j+1 # pulamos a primeira coluna

        razao = T[1:, -1]/T[1:, j] # divide os valores de b pela coluna (máximo de aumento da variável da coluna k)
        razao = np.where(razao < 0, np.nan_to_num(np.inf), razao) # substitui valores negativo por +inf

        i = np.argmin(razao) # obtem o índice do menor valor (menor dentre os máximos para a variável da coluna k)
        i = i+1 # pulamos a primeira linha

        T[i,:] = T[i,:]/T[i, j] # dividindo a linha pelo valor escolhido

        for l in range(T.shape[0]):

            if l != i:
                m = T[l, j]/T[i,j] # fator multiplicativo para zerar o elemento
                T[l,:] = T[l,:] - m*T[i,:]

            else:
                T[i,j] = T[i,j]

        counter +=1
        print('j: {}'.format(j))
        print('i: {}'.format(i))
        print('T: {}\n'.format(T.round(2)))
      
    return T

### Exemplos

* Problema 2.5 (1)

In [385]:
A = np.array([[1, 2, -2, 0],
              [0, 1, 3, 1]])
b = np.array([2,5]).reshape(2,1)
c = np.array([0, 3, 1, 0]).reshape(4,1)
B = [0, 3]

In [386]:
SimplexTableau(A, b, c, B, max_counter=20)

>>> Iteração 0:
j: 2
i: 1
T: [[ 1.   1.5  0.  -4.   0.   3. ]
 [ 0.   0.5  1.  -1.   0.   1. ]
 [ 0.  -0.5  0.   4.   1.   4. ]]

>>> Iteração 1:
j: 3
i: 2
T: [[ 1.     1.     0.     0.     1.     7.   ]
 [ 0.     0.375  1.     0.     0.25   2.   ]
 [ 0.    -0.125  0.     1.     0.25   1.   ]]



array([[ 1.   ,  1.   ,  0.   ,  0.   ,  1.   ,  7.   ],
       [ 0.   ,  0.375,  1.   ,  0.   ,  0.25 ,  2.   ],
       [ 0.   , -0.125,  0.   ,  1.   ,  0.25 ,  1.   ]])

Checando a solução (determinada pelas colunas $j$ binárias do *tableau*, onde $x_j$ assume o valor da última coluna):

In [387]:
x = np.array([0, 2, 1, 0]).reshape(4, 1)

In [388]:
np.dot(A, x)

array([[2],
       [5]])

In [389]:
b # Ax = b!

array([[2],
       [5]])

In [390]:
np.dot(c.T, x) # valor máximo de z

array([[7]])

* Exemplo da seção *2.3 A simplex iteration*

In [426]:
A = np.array([[1,1,1,0,0], 
               [2,1,0,1,0], 
               [-1,1,0,0,1]])
c = np.array([2,3,0,0,0]).reshape(5,1)
b = np.array([6,10,4]).reshape(3,1)
B = [2, 3, 4]

In [427]:
T = SimplexTableau(A, b, c, B, max_counter=20)

>>> Iteração 0:
j: 1
i: 2
T: [[ 1.   0.  -2.   0.   1.   0.  10. ]
 [ 0.   0.   0.5  1.  -0.5  0.   1. ]
 [ 0.   1.   0.5  0.   0.5  0.   5. ]
 [ 0.   0.   1.5  0.   0.5  1.   9. ]]

>>> Iteração 1:
j: 2
i: 1
T: [[ 1.  0.  0.  4. -1.  0. 14.]
 [ 0.  0.  1.  2. -1.  0.  2.]
 [ 0.  1.  0. -1.  1.  0.  4.]
 [ 0.  0.  0. -3.  2.  1.  6.]]

>>> Iteração 2:
j: 4
i: 3
T: [[ 1.   0.   0.   2.5  0.   0.5 17. ]
 [ 0.   0.   1.   0.5  0.   0.5  5. ]
 [ 0.   1.   0.   0.5  0.  -0.5  1. ]
 [ 0.   0.   0.  -1.5  1.   0.5  3. ]]



Checando a solução (determinada pelas colunas $j$ binárias do *tableau*, onde $x_j$ assume o valor da última coluna):

In [396]:
x = np.array([1, 5, 0, 3, 0]).reshape(5, 1)

In [397]:
np.dot(A, x)

array([[ 6],
       [10],
       [ 4]])

In [398]:
b # Ax = b!

array([[ 6],
       [10],
       [ 4]])

In [399]:
np.dot(c.T, x) # valor máximo de z

array([[17]])

### Passo a passo

> Como foi criada a implementação apresentada

Utilizando o exemplo da seção *2.7.2 Tableau* para construir o tableu passo a passo:

In [437]:
A = np.array([[1, -1, 2, -1, 0],
              [2, 0, 1, -1, 1]])
c = np.array([1, -2, 0, 1, 3]).reshape(5,1)
b = np.array([1, -1]).reshape(2,1)

In [438]:
B = np.array([0,3]) # indexação começa em 0! Equivale a B = [1, 4]

Passar o problema para a forma canônica:

In [439]:
def canonical_form(A, b, c, B=None, w=0):
    """
    Passa o PL para a forma canônica: (i) A_B deve ser identidade, (ii) c_B = 0
    """

    if np.all(B != None):
        if not np.all(c[B] != 0) or not np.all(A[:, B] != np.identity(len(B))):

            A_Binv = np.linalg.inv(A[:, B])
            y = A_Binv.T.dot(c[B])
            c = c.T - y.T.dot(A).ravel()
            c = c.T
            w = w + np.dot(y.T, b)

            A = A_Binv.dot(A)
            b = A_Binv.dot(b)

            print('A: \n', A)
            print('b: \n', b)
            print('c: \n', c)
            print('w: \n', w)
            print(">> Problema canonizado!")
            return A, b, c, w

        else:
            print(">> Problema ja na forma canonica!")
            return
    
    else:
        pass

In [440]:
A, b, c, w = canonical_form(A, b, c, B)

A: 
 [[ 1.  1. -1.  0.  1.]
 [ 0.  2. -3.  1.  1.]]
b: 
 [[-2.]
 [-3.]]
c: 
 [[ 0.]
 [-5.]
 [ 4.]
 [ 0.]
 [ 1.]]
w: 
 [[-5.]]
>> Problema canonizado!


#### Construindo o *tableu* inicial

Na tabela formada, as linhas correspondem a variáveis não básicas, cujos valroes estão determinados na última coluna.

In [441]:
T = np.concatenate((np.zeros(A.shape[0]).reshape(A.shape[0],1), A, b), axis=1)
T = np.concatenate((np.array([1] + list(-c.T[0]) + [0]).reshape(1, c.shape[0]+2), T), axis=0)
T

array([[ 1., -0.,  5., -4., -0., -1.,  0.],
       [ 0.,  1.,  1., -1.,  0.,  1., -2.],
       [ 0.,  0.,  2., -3.,  1.,  1., -3.]])

#### *Pivot* - trocando uma variável básica por uma não básica

Aplicando o procedimento do *pivot* num elemento $(i,j)$ do tableu T significa gerar o tableu T' tal que:

<img src="files/pivoting.png" width="350">

Escolhemos o elemento $(i, j)$ tal que $- c_j < 0$ e $j = argmin (\frac{b_i}{T(i,k)} : \frac{b_i}{T(i,k)} > 0)$:

In [442]:
j = np.where(T[0,1:-1] < 0)[0][0] # primeiro elemento em quem -c > 0 (exceto primeira e última colunas)
j = j+1 # pulamos a primeira coluna
T[:, j]

array([-4., -1., -3.])

In [443]:
razao = T[1:, -1]/T[1:, j] # divide os valores de b pela coluna (máximo de aumento da variável da coluna k)
razao

array([2., 1.])

In [444]:
razao = np.where(razao < 0, np.nan_to_num(np.inf), razao) # substitui valores negativo por +inf
i = np.argmin(razao) # obtem o índice do menor valor (menor dentre os máximos para a variável da coluna k)
i = i+1 # pulamos a primeira linha
i

2

O elemento escolhido foi:

In [445]:
print("T({},{}) = {}".format(i, j, T[i, j]))

T(2,3) = -3.0


In [446]:
T[i,:] = T[i,:]/T[i, j] # dividindo a linha pelo valor escolhido
T.round(2)

array([[ 1.  , -0.  ,  5.  , -4.  , -0.  , -1.  ,  0.  ],
       [ 0.  ,  1.  ,  1.  , -1.  ,  0.  ,  1.  , -2.  ],
       [-0.  , -0.  , -0.67,  1.  , -0.33, -0.33,  1.  ]])

In [447]:
for l in range(T.shape[0]):
    
    if l != i:
        m = T[l, j]/T[i,j] # fator multiplicativo para zerar o elemento
        T[l,:] = T[l,:] - m*T[i,:]
        
    else:
        T[i,j] = T[i,j]

In [448]:
T.round(2)

array([[ 1.  , -0.  ,  2.33,  0.  , -1.33, -2.33,  4.  ],
       [ 0.  ,  1.  ,  0.33,  0.  , -0.33,  0.67, -1.  ],
       [-0.  , -0.  , -0.67,  1.  , -0.33, -0.33,  1.  ]])

#### Repetindo o procedimento

Agora que temos novas variáveis básicas (referentes às colunas 3 e 4), repetimos o procedimento. 

Temos uma solução determinada quando todos $-c >= 0$, ou seja, a primeira linha se tornar não negativa. 

In [459]:
counter = 0

while np.any(T[0,:] < 0) and (counter < 10):
    
    print('>>> Iteração {}:'.format(counter))
    
    j = np.where(T[0,1:-1] < 0)[0][0] # primeiro elemento em quem -c > 0 (exceto primeira e última colunas)
    j = j+1 # pulamos a primeira coluna
    
    razao = T[1:, -1]/T[1:, j] # divide os valores de b pela coluna (máximo de aumento da variável da coluna k)
    razao = np.where(razao < 0, np.nan_to_num(np.inf), razao) # substitui valores negativo por +inf
    
    i = np.argmin(razao) # obtem o índice do menor valor (menor dentre os máximos para a variável da coluna k)
    i = i+1 # pulamos a primeira linha

    T[i,:] = T[i,:]/T[i, j] # dividindo a linha pelo valor escolhido
    
    for l in range(T.shape[0]):

        if l != i:
            m = T[l, j]/T[i,j] # fator multiplicativo para zerar o elemento
            T[l,:] = T[l,:] - m*T[i,:]

        else:
            T[i,j] = T[i,j]
        
    print('T: {}\n'.format(T.round(2)))
            
    counter +=1

>>> Iteração 0:
T: [[ 1.    0.    2.33  0.   -1.33 -2.33  4.  ]
 [ 0.    1.    0.33  0.   -0.33  0.67 -1.  ]
 [ 0.    0.   -0.67  1.   -0.33 -0.33  1.  ]]

>>> Iteração 1:
T: [[ 1. -4.  1.  0.  0. -5.  8.]
 [-0. -3. -1. -0.  1. -2.  3.]
 [ 0. -1. -1.  1.  0. -1.  2.]]

>>> Iteração 2:
T: [[ 1.    0.    2.33  0.   -1.33 -2.33  4.  ]
 [ 0.    1.    0.33  0.   -0.33  0.67 -1.  ]
 [ 0.    0.   -0.67  1.   -0.33 -0.33  1.  ]]

>>> Iteração 3:
T: [[ 1. -4.  1.  0.  0. -5.  8.]
 [-0. -3. -1. -0.  1. -2.  3.]
 [ 0. -1. -1.  1.  0. -1.  2.]]

>>> Iteração 4:
T: [[ 1.    0.    2.33  0.   -1.33 -2.33  4.  ]
 [ 0.    1.    0.33  0.   -0.33  0.67 -1.  ]
 [ 0.    0.   -0.67  1.   -0.33 -0.33  1.  ]]

>>> Iteração 5:
T: [[ 1. -4.  1.  0.  0. -5.  8.]
 [-0. -3. -1. -0.  1. -2.  3.]
 [ 0. -1. -1.  1.  0. -1.  2.]]

>>> Iteração 6:
T: [[ 1.    0.    2.33  0.   -1.33 -2.33  4.  ]
 [ 0.    1.    0.33  0.   -0.33  0.67 -1.  ]
 [ 0.    0.   -0.67  1.   -0.33 -0.33  1.  ]]

>>> Iteração 7:
T: [[ 1. -4.  1.  

O problema está cíclico! Talvez seja ilimitado, vamos checar:

In [461]:
from scipy.optimize import linprog

In [462]:
linprog(c, A_eq=A, b_eq=b)

     con: array([0., 0.])
     fun: 8.0
 message: 'Optimization failed. The problem appears to be unbounded.'
     nit: 2
   slack: array([], dtype=float64)
  status: 3
 success: False
       x: array([0., 0., 2., 3., 0.])